In [1]:
import pysftp

In [52]:
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None
hostname = 'mlp-trigger-1.sh.intel.com'
username = 'gta'
password = 'gta'
# upload file
# a=r'C:\Users\xinfahux\OneDrive - Intel Corporation\Desktop\record.txt'
b='/home/gta/pysftp/record.txt'

#download file
# a=r'C:\Users\xinfahux\OneDrive - Intel Corporation\Desktop\helper.py'
# b='/home/gta/python3.7/framework.ai.validation.ipex-gpu-nightly/common/helper.py'
sftp = pysftp.Connection(host=hostname, username=username, password=password, cnopts=cnopts)
#sftp.put(a,b)
#sftp.get(b,a)
#sftp.remove(b)#删除文件：如果远程没有这个文件则会报错
remotepath = r'/home/gta/whls/ww10_00001'

s = sftp.mkdir(remotepath)
sftp.put(r'dist\asd.whl')
sftp.close()

In [48]:
import time
s='saee'
'WW' + str(int(time.strftime('%W')) + 1) + '_{}'.format(s)

'WW10_saee'

In [22]:
from datetime import datetime
datetime.now().isocalendar()

(2022, 9, 1)

In [26]:
time.localtime()[7]/7+1

9.428571428571429

In [ ]:
import pysftp
import time
import os
import argparse
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None
hostname = 'mlp-trigger-1.sh.intel.com'
username = 'gta'
password = 'gta'

def init_args():
    parser = argparse.ArgumentParser(description=f"send file to remote machine")
    parser.add_argument('--cur_cmt', '-c', required=True, help='current commit id')
    parser.add_argument('--filename', '-f', required=True, help='build produces a file name')
    args = parser.parse_args()
    return args

def upload_file(args):
    with pysftp.Connection(host=hostname, username=username, password=password, cnopts=cnopts) as sftp:
        dirname = 'WW{}_{}'.format(str(int(time.strftime('%W')) + 1), args.cur_cmt)
        remote_dirpath = f'/home/gta/whls/{dirname}'
        if not sftp.exists(remote_dirpath):
            sftp.mkdir(remote_dirpath)
            sftp.put(args.filename, os.path.join(remote_dirpath, args.filename))

if __name__ == '__main__':
    args = init_args()
    upload_file(args)

In [59]:
python -c "import pysftp; import time; import os; cnopts = pysftp.CnOpts(); cnopts.hostkeys = None; current_commit = os.environ.get('current_commit'); filename = os.environ.get('filename'); dirname = 'WW{}_{}'.format(str(int(time.strftime('%W')) + 1), current_commit); remote_dirpath = f'/home/gta/whls/{dirname}'; sftp = pysftp.Connection(host='mlp-trigger-1.sh.intel.com', username='gta', password='gta', cnopts=cnopts); sftp.mkdir(remote_dirpath) if not sftp.exists(remote_dirpath) else print('remote_dirpath exists'); sftp.put(filename, os.path.join(remote_dirpath, filename)); sftp.close()"

SyntaxError: invalid syntax (<ipython-input-59-0f504af8e2bc>, line 2)

In [61]:
import os

In [ ]:
def build_gpu(machine, py_version){
    map_dict = {
        node(machine){
            cleanWs()
            stage("torch source"){
                println("================================================================================")
                println("get torch source code")
                println("================================================================================")
                withEnv(["py_version=${py_version}"]){
                    sh '''
                        set -e
                        set +x
                        mkdir ${WORKSPACE}/python${py_version} && cd ${WORKSPACE}/python${py_version}
                        git clone -b pytorch-1.7 --depth=1 https://github.com/intel-innersource/frameworks.ai.pytorch.private-gpu.git
                    '''
                }
            }
            stage("install torch"){
                println("================================================================================")
                println("install torch")
                println("================================================================================")
                withEnv(["py_version=${py_version}"]){
                    sh '''
                        set -e
                        set +x
                        if [ -e ${HOME}/miniconda3/etc/profile.d/conda.sh ]; then
                            source ${HOME}/miniconda3/etc/profile.d/conda.sh 2>&1 >>/dev/null
                        else
                            echo -e "[ Error ] No Conda Found!"
                            exit 1
                        fi
                        env_path=`find ${HOME}/miniconda3/envs -type f -name python${py_version}`
                        if [ -z $env_path ];then
                            echo -e "[ INFO ] Create a virtual environment named py${py_version} and apply it"
                            conda create -y --name py${py_version} python=${py_version}
                            conda activate py${py_version}
                        else
                            path_arr=(`echo $env_path | tr '/' ' '`)
                            conda activate ${path_arr[-3]}
                        fi
                        export LD_LIBRARY_PATH=${HOME}/intel/oneapi/mkl/latest/lib:${HOME}/intel/oneapi/mkl/latest/lib64:${HOME}/intel/oneapi/mkl/latest/lib/intel64:${LD_LIBRARY_PATH}
                        installed_commit=$(python -c "import torch;print(torch.version.git_version[:5])")
                        cd ${WORKSPACE}/python${py_version}/frameworks.ai.pytorch.private-gpu
                        current_commit="$(git rev-parse --short=5 HEAD)"
                        echo -e "[ INFO ] current torch commit : $current_commit"
                        echo -e "[ INFO ] installed torh commit: $installed_commit"

                        if [ "$current_commit" == "$installed_commit" ]; then
                            echo -e "[ INFO ] PyTorch is already the latest one"
                        else
                            echo -e "[ INFO ] Build PyTorch form source"
                            git submodule update --init --recursive
                            python -m pip uninstall -y pybind11
                            python -m pip install -r requirements.txt
                            INTEL_MKL_DIR=${HOME}/intel/oneapi/mkl/latest python ./setup.py bdist_wheel
                            python -m pip install dist/*.whl
                            if [ $? -ne 0 ]; then
                                echo -e "[ERROR] Build PyTorch Failed"
                                exit 1
                            fi
                        fi
                        conda deactivate
                    '''
                }


            }

            stage("ipex source"){
                println("================================================================================")
                println("get ipex source code")
                println("================================================================================")
                withEnv(["py_version=${py_version}"]){
                    sh '''
                        set -e
                        set +x
                        if [ -e ${WORKSPACE}/python${py_version} ] && [ -d ${WORKSPACE}/python${py_version} ]; then
                           cd ${WORKSPACE}/python${py_version}
                        else
                            mkdir ${WORKSPACE}/python${py_version} && cd ${WORKSPACE}/python${py_version}
                        fi
                        git clone https://github.com/intel-innersource/frameworks.ai.pytorch.ipex-gpu.git
                        cd frameworks.ai.pytorch.ipex-gpu
                        git pull origin ${ghprbSourceBranch}
                        if [ $? -ne 0 ]; then
                            echo -e "[ERROR] Merge Code ${ghprbSourceBranch} --> master Failed"
                            exit 1
                        fi
                        git submodule sync
                        git submodule update --init --recursive
                    '''
                }
            }
            stage("code lint"){
                println("================================================================================")
                println("check code style")
                println("================================================================================")
                withEnv(["py_version=${py_version}"]){
                    sh '''
                        set -e
                        set +x
                        cd ${WORKSPACE}/python${py_version}/frameworks.ai.pytorch.ipex-gpu
                        if [ -e ${HOME}/miniconda3/etc/profile.d/conda.sh ]; then
                            source ${HOME}/miniconda3/etc/profile.d/conda.sh 2>&1 >>/dev/null
                        else
                            echo -e "[ Error ] No Conda Found!"
                            exit 1
                        fi
                        env_path=`find ${HOME}/miniconda3/envs -type f -name python${py_version}`
                        if [ -z $env_path ];then
                            echo -e "[ INFO ] Create a virtual environment named py${py_version} and apply it"
                            conda create -y --name py${py_version} python=${py_version}
                            conda activate py${py_version}
                        else
                            path_arr=(`echo $env_path | tr '/' ' '`)
                            conda activate ${path_arr[-3]}
                        fi
                        find csrc ipex/csrc -name "*.c" -o -name "*.cpp" -o -name "*.h" -o -name "*.hpp" | grep -v "generated" | xargs clang-format --dry-run -style=file --Werror 2>&1 | tee ./check_clang_format.log
                        if [ -s ../check_clang_format.log ]; then
                            echo "[ERROR] clang-format lint check Failed"
                            exit 1
                        fi
                        python -m pip install flake8 || true
                        python -m flake8 2>&1 | tee ./check_flake8.log
                        if [ -s ./check_flake8.log ]; then
                            echo "[ERROR] flake8 lint check Failed"
                            exit 1
                        fi
                    '''
                }

            }
            stage("install ipex"){
                println("================================================================================")
                println("install ipex")
                println("================================================================================")
                withEnv(["py_version=${py_version}"]){
                    sh '''
                        set -e
                        set +x
                        if [ -e ${HOME}/miniconda3/etc/profile.d/conda.sh ]; then
                            source ${HOME}/miniconda3/etc/profile.d/conda.sh 2>&1 >>/dev/null
                        else
                            echo -e "[ Error ] No Conda Found!"
                            exit 1
                        fi
                        env_path=`find ${HOME}/miniconda3/envs -type f -name python${py_version}`
                        if [ -z $env_path ];then
                            echo -e "[ INFO ] Create a virtual environment named py${py_version} and apply it"
                            conda create -y --name py${py_version} python=${py_version}
                            conda activate py${py_version}
                        else
                            path_arr=(`echo $env_path | tr '/' ' '`)
                            conda activate ${path_arr[-3]}
                        fi
                        [ -e ${HOME}/env.sh ] && source ${HOME}/env.sh 2>&1 >>/dev/null
                        cd ${WORKSPACE}/python${py_version}/frameworks.ai.pytorch.ipex-gpu
                        python -m pip uninstall -y ipex || true
                        python -m pip install -r requirements.txt || true
                        git submodule update --init --recursive
                        python setup.py bdist_wheel
                        python -m pip install dist/*.whl
                        if [ $? -ne 0 ]; then
                            echo -e "[ERROR] Build IPEX Failed"
                            exit 1
                        fi

                    '''
                }//withEnv
            }
            stage("move wheel"){
                println("================================================================================")
                println("move wheel")
                println("================================================================================")
                withEnv(["py_version=${py_version}"]){
                    sh '''
                        set -e
                        set +x
                        cd ${WORKSPACE}/python${py_version}/frameworks.ai.pytorch.ipex-gpu/dist
                        python -m pip install pysftp
                        export filename=`ls`
                        export current_commit =`git log -1 --oneline | awk '{print $1}'`
                        python -c "import pysftp; import time; import os; cnopts = pysftp.CnOpts(); cnopts.hostkeys = None; hostname = 'mlp-trigger-1.sh.intel.com'; username = 'gta'; password = 'gta'; current_commit = os.environ.get('current_commit'); filename = os.environ.get('filename'); dirname = 'WW{}_{}'.format(str(int(time.strftime('%W')) + 1), current_commit); remote_dirpath = f'/home/gta/whls/{dirname}'; sftp = pysftp.Connection(host=hostname, username=username, password=password, cnopts=cnopts); sftp.mkdir(remote_dirpath) if not sftp.exists(remote_dirpath) else print('remote_dirpath exists'); sftp.put(filename, os.path.join(remote_dirpath, filename)); sftp.close()"
                    '''
                }//withEnv
            }//stage
        }//node
    };//map
    return map_dict;
}
def parallel_tasks = [:]
parallel_tasks["python3.7"] = build_gpu("mlp-pytorchdgpu-build", 3.7)
parallel_tasks["python3.8"] = build_gpu("mlp-pytorchdgpu-build", 3.8)
parallel_tasks["python3.9"] = build_gpu("mlp-pytorchdgpu-build", 3.9)
parallel(parallel_tasks)


In [68]:
import json
d1={
    'a':20,
    'b':30,
    'c':90,
    'name':'xinfahuan'
}
with open('pyload.json','w') as f:
    print(json.dumps(d1))

{"a": 20, "b": 30, "c": 90, "name": "xinfahuan"}


In [ ]:
a=['test_case', 'priority', 'function_status', 'acc_status', 'exp_acc', 'ref_acc', 'cur_acc', 'perf_status', 'exp_perf', 'ref_perf', 'cur_perf', unit, ref_log]

In [ ]:
function_status = None
        unit = None
        result = [test_case, priority, function_status, acc_status, exp_acc, ref_acc, cur_acc, perf_status, exp_perf, ref_perf, cur_perf, unit, ref_log]